In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import json

In [2]:
df = pd.read_csv('../../data/illinois_ambient_annual_loads_wrtdsk_raw.csv', dtype={'site_no':str}).rename({'_flow_days':'flow_days'}, axis=1)

#df['year'] = pd.to_datetime(df['year'], format='%Y').values
#df = df.set_index(['site_no','year'])

df.head()

,site_no,year,00530_kg,00535_kg,00600_kg,00610_kg,00625_kg,00630_kg,00665_kg,00666_kg,...,00610_n,00625_n,00630_n,00665_n,00666_n,00667_n,00946_n,80154_n,99220_n,flow_days
0,03336645,1979,4.441585e+08,NaN,5247228.68,NaN,2389157.45,3804656.32,210173.74,NaN,...,0,12,12,12,0,0,0,0,0,365
1,03336645,1980,2.823931e+08,NaN,2893754.65,NaN,1028470.86,2028750.03,88583.75,NaN,...,0,13,13,5,0,0,0,0,13,366
2,03336645,1981,1.886789e+08,NaN,4858619.78,NaN,977276.77,3475216.04,NaN,38629.77,...,0,12,9,0,9,0,0,0,12,365
3,03336645,1982,5.648412e+08,NaN,6185522.08,NaN,2942128.68,4068385.10,NaN,85213.76,...,0,8,8,0,8,0,0,0,8,365
4,03336645,1983,9.445388e+07,NaN,3856953.07,NaN,730680.37,3071834.27,NaN,31662.84,...,0,8,8,0,8,0,0,0,8,365


In [3]:
# data cleaning 00535_kg
param='00535'
site = '03384450'
years = [2015]

bad_index = (df.site_no == site) & (df.year.isin(years))
df.loc[bad_index,[f'{param}_kg',f'{param}_gse']] = np.nan
df.loc[bad_index]

,site_no,year,00530_kg,00535_kg,00600_kg,00610_kg,00625_kg,00630_kg,00665_kg,00666_kg,...,00610_n,00625_n,00630_n,00665_n,00666_n,00667_n,00946_n,80154_n,99220_n,flow_days
510,03384450,2015,4374649.7,NaN,62897.72,NaN,NaN,14928.37,49137.81,NaN,...,0,0,11,11,0,0,9,0,9,365


In [4]:
# data cleaning for 00625_kg
param='00625'
site = '03382100'
years = [2012,2013,2018,2019]

bad_index = (df.site_no == site) & (df.year.isin(years))
df.loc[bad_index,[f'{param}_kg',f'{param}_gse']] = np.nan
df.loc[bad_index]

,site_no,year,00530_kg,00535_kg,00600_kg,00610_kg,00625_kg,00630_kg,00665_kg,00666_kg,...,00610_n,00625_n,00630_n,00665_n,00666_n,00667_n,00946_n,80154_n,99220_n,flow_days
464,03382100,2012,5907332.87,1178232.84,185301.02,14238.84,NaN,91330.03,12292.57,NaN,...,7,7,9,9,0,0,9,0,9,366
465,03382100,2013,10799231.55,1492259.62,145874.43,13606.93,NaN,36469.13,21436.67,NaN,...,9,9,9,9,0,0,9,0,9,365
470,03382100,2018,17523855.00,1817083.98,192064.74,12368.43,NaN,54063.06,31419.50,NaN,...,9,9,9,9,0,0,9,0,9,365
471,03382100,2019,27015384.52,2918228.89,296756.43,20974.66,NaN,82016.98,39896.80,NaN,...,9,9,9,9,0,0,9,0,9,365


In [5]:
# drop years prior to 1976
df = df[df['year'] >= 1976]

In [6]:
df = df.rename({'site_no':'site'}, axis=1).set_index(['site','year'])

In [7]:
#df.groupby('site').min()['year'].tolist()

# rename columns

In [8]:
df.columns = [col.replace('_n','_count') for col in df.columns]
units = [col.split('_')[1] for col in df.columns]
params = [col.split('_')[0] for col in df.columns] #if col.split('_')[0]!='flow
#params

In [9]:
mapper = {
          'site_no':'site',
          'year' : 'year',
          '00530':'total suspended solids',
          '00535':'volatile suspended solids',
          '00600':'total nitrogen',
          '00610':'ammonia nitrogen',
          '00625':'organic nitrogen',
          '00630':'nitrate nitrogen',
          '00665':'total phosphorus',
          '00666':'dissolved phosphorus',
          '00667':'particulate phosphorus',
          '00946':'sulfate',
          '80154':'suspended sediment',
          '99220':'chloride',
          'flow':'flow'}

In [10]:
col_names = []
for i, col in enumerate(df.columns):
    #print(col)
    name = mapper.get(col.split('_')[0])
    unit = units[i]
    col_names.append(f'{name} [{unit}]')
    #col_names.append
    
#col_names
df.columns = col_names
df.head()

total suspended solids [kg]  volatile suspended solids [kg]  \
site     year                                                                
03336645 1979                 4.441585e+08                             NaN   
         1980                 2.823931e+08                             NaN   
         1981                 1.886789e+08                             NaN   
         1982                 5.648412e+08                             NaN   
         1983                 9.445388e+07                             NaN   

               total nitrogen [kg]  ammonia nitrogen [kg]  \
site     year                                               
03336645 1979           5247228.68                    NaN   
         1980           2893754.65                    NaN   
         1981           4858619.78                    NaN   
         1982           6185522.08                    NaN   
         1983           3856953.07                    NaN   

               organic nitrogen [kg]  nitrate nitrogen [kg]  \
site     year                                                 
03336645 1979             2389157.45             3804656.32   
         1980             1028470.86             2028750.03   
         1981              977276.77             3475216.04   
         1982             2942128.68             4068385.10   
         1983              730680.37             3071834.27   

               total phosphorus [kg]  dissolved phosphorus [kg]  \
site     year                                                     
03336645 1979              210173.74                        NaN   
         1980               88583.75                        NaN   
         1981                    NaN                   38629.77   
         1982                    NaN                   85213.76   
         1983                    NaN                   31662.84   

               particulate phosphorus [kg]  sulfate [kg]  ...  \
site     year                                             ...   
03336645 1979                          NaN           NaN  ...   
         1980                          NaN           NaN  ...   
         1981                          NaN           NaN  ...   
         1982                          NaN           NaN  ...   
         1983                          NaN           NaN  ...   

               ammonia nitrogen [count]  organic nitrogen [count]  \
site     year                                                       
03336645 1979                         0                        12   
         1980                         0                        13   
         1981                         0                        12   
         1982                         0                         8   
         1983                         0                         8   

               nitrate nitrogen [count]  total phosphorus [count]  \
site     year                                                       
03336645 1979                        12                        12   
         1980                        13                         5   
         1981                         9                         0   
         1982                         8                         0   
         1983                         8                         0   

               dissolved phosphorus [count]  particulate phosphorus [count]  \
site     year                                                                 
03336645 1979                             0                               0   
         1980                             0                               0   
         1981                             9                               0   
         1982                             8                               0   
         1983                             8                               0   

               sulfate [count]  suspended sediment [count]  chloride [count]  \
site     year                                                         

# load flow data

In [38]:
#df = df.reset_index()

flow_df = pd.read_csv('../../data/illinois_ambient_annual_flow.csv', dtype={'site_no':str}).rename({'site_no':'site'}, axis=1).set_index(['site','year'])
flow_df = flow_df.rename({'mean flow [cms]':'mean flow [m^3/sec]'}, axis=1)
out = df.merge(flow_df, left_index=True, right_index=True, how='left')
#out = df.merge(flow_df, on=['site','year'], how='left')
#out = out.set_index(['site','year'])

out.head()

total suspended solids [kg]  volatile suspended solids [kg]  \
site     year                                                                
03336645 1979                 4.441585e+08                             NaN   
         1980                 2.823931e+08                             NaN   
         1981                 1.886789e+08                             NaN   
         1982                 5.648412e+08                             NaN   
         1983                 9.445388e+07                             NaN   

               total nitrogen [kg]  ammonia nitrogen [kg]  \
site     year                                               
03336645 1979           5247228.68                    NaN   
         1980           2893754.65                    NaN   
         1981           4858619.78                    NaN   
         1982           6185522.08                    NaN   
         1983           3856953.07                    NaN   

               organic nitrogen [kg]  nitrate nitrogen [kg]  \
site     year                                                 
03336645 1979             2389157.45             3804656.32   
         1980             1028470.86             2028750.03   
         1981              977276.77             3475216.04   
         1982             2942128.68             4068385.10   
         1983              730680.37             3071834.27   

               total phosphorus [kg]  dissolved phosphorus [kg]  \
site     year                                                     
03336645 1979              210173.74                        NaN   
         1980               88583.75                        NaN   
         1981                    NaN                   38629.77   
         1982                    NaN                   85213.76   
         1983                    NaN                   31662.84   

               particulate phosphorus [kg]  sulfate [kg]  ...  \
site     year                                             ...   
03336645 1979                          NaN           NaN  ...   
         1980                          NaN           NaN  ...   
         1981                          NaN           NaN  ...   
         1982                          NaN           NaN  ...   
         1983                          NaN           NaN  ...   

               organic nitrogen [count]  nitrate nitrogen [count]  \
site     year                                                       
03336645 1979                        12                        12   
         1980                        13                        13   
         1981                        12                         9   
         1982                         8                         8   
         1983                         8                         8   

               total phosphorus [count]  dissolved phosphorus [count]  \
site     year                                                           
03336645 1979                        12                             0   
         1980                         5                             0   
         1981                         0                             9   
         1982                         0                             8   
         1983                         0                             8   

               particulate phosphorus [count]  sulfate [count]  \
site     year                                                    
03336645 1979                               0                0   
         1980                               0                0   
         1981                               0                0   
         1982                               0                0   
         1983                               0                0   

               suspended sediment [count]  chloride [count]  flow [days]  \
site     year                                                              
03336645 1979                           0                 0  

# compute metadata

In [41]:
import xarray as xr
from meta import *

In [42]:
#df = df.set_index(['site','datetime'])
ds = out.to_xarray()
ds

<xarray.Dataset>
Dimensions:                            (site: 82, year: 46)
Coordinates:
  * site                               (site) object '03336645' ... '05599490'
  * year                               (year) int64 1976 1977 1978 ... 2020 2021
Data variables: (12/38)
    total suspended solids [kg]        (site, year) float64 nan ... 1.439e+08
    volatile suspended solids [kg]     (site, year) float64 nan nan ... nan nan
    total nitrogen [kg]                (site, year) float64 nan ... 2.622e+06
    ammonia nitrogen [kg]              (site, year) float64 nan ... 2.155e+05
    organic nitrogen [kg]              (site, year) float64 nan nan ... 1.91e+06
    nitrate nitrogen [kg]              (site, year) float64 nan ... 7.047e+05
    ...                                 ...
    particulate phosphorus [count]     (site, year) float64 nan nan ... 4.0 4.0
    sulfate [count]                    (site, year) float64 nan nan ... 4.0 4.0
    suspended sediment [count]         (site, year) float64 nan nan ... 0.0 0.0
    chloride [count]                   (site, year) float64 nan nan ... 4.0 4.0
    flow [days]                        (site, year) float64 nan nan ... 365.0
    mean flow [m^3/sec]                (site, year) float64 nan nan ... 69.65

In [43]:
#meta_file = 'https://www.sciencebase.gov/catalog/file/get/60abcec0d34ea221ce51e8a3?f=__disk__92%2F30%2Fb7%2F9230b7cfe0ad43e5eb1e36353a135d0bb6d824be'
meta_file = 'illinois_ambient_annual_loads_wrtdsk_metadata.xml'
import_metadata(ds, meta_file)

In [45]:
update_metadata(ds)

In [46]:
out = ds.to_dataframe()

In [47]:
out = out.dropna(how='all')
#out.reset_index().year.value_counts().sort_index()

In [48]:
out.to_csv('../../data/illinois_ambient_annual_loads_wrtdsk.csv')

In [49]:
dump_metadata(ds, '../../data/llinois_ambient_annual_loads_wrtdsk_metadata.xml')

# Scratch